In [6]:
!pip install tensorflow-addons

     -------------------------------------- 746.7/746.7 kB 4.7 MB/s eta 0:00:00


In [16]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# 1. Pré-processar os dados
# Carregue seus datasets balanceados
train_data = pd.read_csv('../data/preprocessed/movies_genres_train_preprocessed.csv')
test_data = pd.read_csv('../data/preprocessed/movies_genres_test_preprocessed.csv')

category_columns = train_data.columns.drop(['Name', 'Description', 'Combined'])

# Defina os parâmetros de pré-processamento
max_features = 10000  # Número máximo de palavras a serem usadas (palavras mais frequentes)
maxlen = 200  # Número máximo de palavras no texto

# Tokenize os textos
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_data['Combined'])

X_train = tokenizer.texts_to_sequences(train_data['Combined'])
X_test = tokenizer.texts_to_sequences(test_data['Combined'])

# Padronize os textos
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

# Obtenha os labels one-hot encoded
y_train = train_data[category_columns].values
y_test = test_data[category_columns].values

# 2. Construir o modelo
embedding_dim = 128  # Dimensão do vetor de embedding
lstm_units = 64  # Unidades LSTM

model = Sequential([
    Embedding(max_features, embedding_dim, input_length=maxlen),
    Bidirectional(LSTM(lstm_units, return_sequences=True)),
    Bidirectional(LSTM(lstm_units)),
    Dense(len(category_columns), activation='sigmoid')
])

# 3. Treinar o modelo
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

batch_size = 32
epochs = 10

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# 4. Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

Epoch 1/10
122/122 [==============================] - 117s 878ms/step - loss: 0.3172 - accuracy: 0.2628 - val_loss: 0.2735 - val_accuracy: 0.3225
Epoch 2/10
122/122 [==============================] - 104s 853ms/step - loss: 0.2853 - accuracy: 0.2647 - val_loss: 0.2740 - val_accuracy: 0.3225
Epoch 3/10
122/122 [==============================] - 101s 826ms/step - loss: 0.2836 - accuracy: 0.2652 - val_loss: 0.2693 - val_accuracy: 0.3295
Epoch 4/10
122/122 [==============================] - 102s 837ms/step - loss: 0.2675 - accuracy: 0.3431 - val_loss: 0.2673 - val_accuracy: 0.3480
Epoch 5/10
122/122 [==============================] - 106s 872ms/step - loss: 0.2437 - accuracy: 0.4304 - val_loss: 0.2570 - val_accuracy: 0.4130
Epoch 6/10
122/122 [==============================] - 107s 874ms/step - loss: 0.2189 - accuracy: 0.4975 - val_loss: 0.2571 - val_accuracy: 0.3921
Epoch 7/10
122/122 [==============================] - 109s 895ms/step - loss: 0.1943 - accuracy: 0.5438 - val_loss: 0.2583 -

In [17]:
from sklearn.metrics import f1_score, average_precision_score, precision_recall_curve

y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
precision_micro = precision_score(y_test, y_pred, average='micro')
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_micro = recall_score(y_test, y_pred, average='micro')
recall_macro = recall_score(y_test, y_pred, average='macro')
print("F1-score (micro): {:.2f}".format(f1_micro))
print("F1-score (macro): {:.2f}".format(f1_macro))
print("Precision (micro): {:.2f}".format(precision_micro))
print("Precision (macro): {:.2f}".format(precision_macro))
print("Recall (micro): {:.2f}".format(recall_micro))
print("Recall (macro): {:.2f}".format(recall_macro))

y_pred = model.predict(X_test)
f1_scores_per_class = f1_score(y_test, np.round(y_pred), average=None)
auc_pr_per_class = average_precision_score(y_test, y_pred, average=None)
for i, category in enumerate(category_columns):
    print("Class: {}".format(category))
    print("F1-score: {:.2f}".format(f1_scores_per_class[i]))
    print("AUC-PR: {:.2f}".format(auc_pr_per_class[i]))
    print("\n")

45/45 [==============================] - 17s 309ms/step


C:\Users\carol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1-score (micro): 0.45
F1-score (macro): 0.21
Precision (micro): 0.56
Precision (macro): 0.33
Recall (micro): 0.37
Recall (macro): 0.18
45/45 [==============================] - 14s 305ms/step
Class: Action
F1-score: 0.41
AUC-PR: 0.50


Class: Adventure
F1-score: 0.46
AUC-PR: 0.47


Class: Animation
F1-score: 0.16
AUC-PR: 0.23


Class: Biography
F1-score: 0.00
AUC-PR: 0.14


Class: Comedy
F1-score: 0.55
AUC-PR: 0.57


Class: Crime
F1-score: 0.51
AUC-PR: 0.54


Class: Documentary
F1-score: 0.23
AUC-PR: 0.16


Class: Drama
F1-score: 0.68
AUC-PR: 0.70


Class: Family
F1-score: 0.18
AUC-PR: 0.18


Class: Fantasy
F1-score: 0.00
AUC-PR: 0.14


Class: History
F1-score: 0.00
AUC-PR: 0.14


Class: Horror
F1-score: 0.28
AUC-PR: 0.36


Class: Music
F1-score: 0.08
AUC-PR: 0.25


Class: Musical
F1-score: 0.00
AUC-PR: 0.11


Class: Mystery
F1-score: 0.00
AUC-PR: 0.21


Class: Romance
F1-score: 0.31
AUC-PR: 0.36


Class: Sci-Fi
F1-score: 0.17
AUC-PR: 0.38


Class: Thriller
F1-score: 0.23
AUC-PR: 0.24
